In [4]:
import os
import shutil
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

# Define paths
original_data_dir = 'C:\\FDS\\AIDER'  
train_path = "C:\\FDS\\Project\\train"
test_path = "C:\\FDS\\Project\\test"
TARGET_SIZE = (256, 256)  

# Split into Train and Test (with resizing)
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

for class_name in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_name)
    images = glob.glob(os.path.join(class_path, "*.jpg"))  
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

    for img_path in train_images:
        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img.save(os.path.join(train_path, class_name, os.path.basename(img_path)))

    for img_path in test_images:
        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img.save(os.path.join(test_path, class_name, os.path.basename(img_path)))
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

def augment_images(class_name, target_count):
    datagen = ImageDataGenerator(
        rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, 
        shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest"
    )

    class_train_path = os.path.join(train_path, class_name)
    images = glob.glob(os.path.join(class_train_path, "*.jpg"))

    current_count = len(images)
    if current_count >= target_count:
        print(f"Skipping {class_name}, already has {current_count} images.")
        return

    print(f"Augmenting {class_name} from {current_count} to {target_count} images...")

    generated = 0  
    images_per_original = max(1, (target_count - current_count) // current_count) 

    for img_path in images:
        if current_count + generated >= target_count:
            break 

        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)

        save_prefix = os.path.basename(img_path).split(".")[0] + "_aug"
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=class_train_path, 
                                  save_prefix=save_prefix, save_format="jpg"):
            generated += 1
            i += 1
            if current_count + generated >= target_count or i >= images_per_original:
                break  

# Apply augmentation to all classes
for class_name in os.listdir(train_path):
    augment_images(class_name, 3200)

print("✅ Augmentation completed successfully!")

Augmenting collapsed_building from 408 to 3200 images...
Augmenting fire from 416 to 3200 images...
Augmenting flooded_areas from 420 to 3200 images...
Skipping normal, already has 3512 images.
Augmenting traffic_incident from 388 to 3200 images...
✅ Augmentation completed successfully!
